To Do:
Write up proper project aim
Small initial analysis with graphs
Then Machine Learning Predictor - predict when in the week I'll go cycling? Give each day a % probability?

## Cycling Schedule Predictor

### Aim
I don't want to say I'm a fair weather cyclist but my days of riding through heavy rain or having to combat brutal headwinds in the name of fitness are reaching an end. Using weather data sourced from the Australian Bureau of Meterology along with my personal cycling data retrieved from Strava (an exercise app), I will develop a model that can review the upcoming weather forecast and predict which days I am most likely to go riding based on my historic rides.

Things I consider: try to go for at least two rides a week, more as I'm training for an event. Rain. Best weather of the upcoming week.


### Data Preparation

In [2]:
# load libraries
import pandas as pd
# import janitor
# for machine learning
# import scikit
# import pytorch
# import XGBoost

In [6]:
# load data
weather = pd.read_csv('Weather Data.csv')
cycling = pd.read_csv('July-Dec Cycling Data.csv')

When looking at the weather forecast, the main information provided is minimum and maximum temperatures, expected rain, wind direction and wind speed. As this is the only information I have access to when deciding whether I'll go riding, any other information can be removed from our data set.

In [13]:
print(weather.columns)
# drop columns
weather = weather.drop(columns=['Daily Weather Observations for Melbourne, Victoria for July - December 2023', 
                               'Evaporation (mm)', 'Direction of maximum wind gust ', 
                               'Speed of maximum wind gust (km/h)', 'Time of maximum wind gust', 
                               '9am relative humidity (%)', '9am cloud amount (oktas)', 
                               '9am MSL pressure (hPa)', '3pm relative humidity (%)', 
                               '3pm cloud amount (oktas)', '3pm MSL pressure (hPa)'])

Index(['Daily Weather Observations for Melbourne, Victoria for July - December 2023',
       'Date', 'Minimum temperature (°C)', 'Maximum temperature (°C)',
       'Rainfall (mm)', 'Evaporation (mm)', 'Sunshine (hours)',
       'Direction of maximum wind gust ', 'Speed of maximum wind gust (km/h)',
       'Time of maximum wind gust', '9am Temperature (°C)',
       '9am relative humidity (%)', '9am cloud amount (oktas)',
       '9am wind direction', '9am wind speed (km/h)', '9am MSL pressure (hPa)',
       '3pm Temperature (°C)', '3pm relative humidity (%)',
       '3pm cloud amount (oktas)', '3pm wind direction',
       '3pm wind speed (km/h)', '3pm MSL pressure (hPa)'],
      dtype='object')


In [14]:
weather.head()
# check data types
# some wind directions are missing and some wind speeds are listed as Calm
# add column with min/max temp in comparison to previous few days, same for wind, same for rain

,Date,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Sunshine (hours),9am Temperature (°C),9am wind direction,9am wind speed (km/h),3pm Temperature (°C),3pm wind direction,3pm wind speed (km/h)
0,2023-07-1,9.7,13.5,0.4,4.2,10.9,W,11,11.3,SW,9
1,2023-07-2,9.6,12.8,0.6,0.2,10.5,SW,6,11.9,SSW,9
2,2023-07-3,8.2,14.5,0.4,0.2,8.4,NNE,7,14.1,N,13
3,2023-07-4,8.3,13.6,0.0,0.0,10.9,NNE,9,13.4,NE,9
4,2023-07-5,10.8,14.7,2.4,3.1,11.6,NW,4,13.7,NNW,13


In [16]:
print(cycling.columns)
# just select the columns I want
cycling = cycling[['Activity Date', 'Moving Time', 'Distance.1', 'Average Speed', 'Elevation Gain',
                   'Average Temperature']]

Index(['Activity ID', 'Activity Date', 'Activity Name', 'Activity Type',
       'Activity Description', 'Elapsed Time', 'Distance', 'Max Heart Rate',
       'Relative Effort', 'Commute', 'Activity Private Note', 'Activity Gear',
       'Filename', 'Athlete Weight', 'Bike Weight', 'Elapsed Time.1',
       'Moving Time', 'Distance.1', 'Max Speed', 'Average Speed',
       'Elevation Gain', 'Elevation Loss', 'Elevation Low', 'Elevation High',
       'Max Grade', 'Average Grade', 'Average Positive Grade',
       'Average Negative Grade', 'Max Cadence', 'Average Cadence',
       'Max Heart Rate.1', 'Average Heart Rate', 'Max Watts', 'Average Watts',
       'Calories', 'Max Temperature', 'Average Temperature',
       'Relative Effort.1', 'Total Work', 'Number of Runs', 'Uphill Time',
       'Downhill Time', 'Other Time', 'Perceived Exertion', 'Type',
       'Start Time', 'Weighted Average Power', 'Power Count',
       'Prefer Perceived Exertion', 'Perceived Relative Effort', 'Commute.1',
    

In [21]:
cycling.head()

,Activity Date,Moving Time,Distance.1,Average Speed,Elevation Gain,Average Temperature,Date,Time
0,"5 Jul 2023, 06:40:41",3704.0,30432.550781,8.216131,30.0,11.0,2023-07-05,06:40:41
1,"10 Jul 2023, 06:55:26",3748.0,30410.199219,8.113714,70.0,13.0,2023-07-10,06:55:26
2,"12 Jul 2023, 07:39:00",4412.0,35607.578125,8.070621,40.0,13.0,2023-07-12,07:39:00
3,"19 Jul 2023, 07:11:12",4264.0,35598.378906,8.348588,40.0,9.0,2023-07-19,07:11:12
4,"21 Jul 2023, 01:48:48",3636.0,31191.509766,8.578523,59.0,13.0,2023-07-21,01:48:48


In [20]:
# check for missing data
# split Activity Date column into Date and Time with Date matching the formatting from weather dataframe.
cycling['Date'] = pd.to_datetime(cycling['Activity Date'], format='%d %b %Y, %H:%M:%S').dt.strftime('%Y-%m-%d')
cycling['Time'] = pd.to_datetime(cycling['Activity Date'], format='%d %b %Y, %H:%M:%S').dt.strftime('%H:%M:%S')
# Move Date and Time columns to front
# drop original Activity Date column

In [ ]:
# merge data sets and create a boolean Ride YN (1/2) column
# create new temperature, wind direction and wind speed columns extracting data from 9am or 3pm depending on cycling Time columns

### Initial Analysis
To analyze cycling habits, including how frequently you ride, how weather conditions affect your riding decisions, and patterns in your cycling behavior over time.

Visuals
How often do I ride, what days do I ride
scatter plot with time and distance coloured by day of week? coloured by temperature?

Features to Consider:

    Weather Data:
        Daily weather conditions
        Historical weather data
    Cycling Data:
        Number of rides per week
        Distance and duration of each ride
        Frequency of rides on specific days of the week
        Rest days
        

### Machine Learning


Modeling Approach:

    Time Series Analysis: Using models like ARIMA or LSTM to predict optimal times based on historical performance data.
    Regression Models: To relate weather conditions at different times of day to your performance metrics.
    Clustering: K-means clustering to identify patterns in your riding times and performance.
    

Modeling Approach:

    Descriptive Statistics:
    Time Series Analysis: Identify trends and patterns in your riding frequency and distances over time.
    Classification Models: Predict likelihood of riding based on past habits and current weather (e.g., logistic regression, decision trees).
    Cluster Analysis: Segment your rides into different types (e.g., short commutes, long weekend rides) and analyze habits within each cluster.

### Results

    Optimal Time Suggestions: Recommendations for the best times to ride based on predicted weather conditions and your historical performance.